In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm


import torch 
import torch.nn as nn
import torch.optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import cv2

import learn2learn as l2l
from learn2learn.data import MetaDataset, FilteredMetaDataset, TaskDataset
from learn2learn.data.transforms import NWays, KShots, LoadData, RemapLabels


In [2]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

device = 'cpu'
if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    device = 'cuda'
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [3]:
labels = {
    "Abdomen": 0,
    "Ankle": 1,
    "Cervical Spine": 2,
    "Chest": 3,
    "Clavicles": 4,
    "Elbow": 5,
    "Feet": 6,
    "Finger": 7,
    "Forearm": 8,
    "Hand": 9,
    "Hip": 10, 
    "Knee": 11,
    "Lower Leg": 12,
    "Lumbar Spine": 13,
    "Others": 14,
    "Pelvis": 15,
    "Shoulder": 16,
    "Sinus": 17,
    "Skull": 18,
    "Thigh": 19,
    "Thoracic Spine": 20,
    "Wrist": 21
}


class XRayDataset(torch.utils.data.Dataset):
    # Dataset for Leap Motion data
    def __init__(self, csv_file, file_col, lbl_col, text2lbl, transforms=None):
        """
        Initialize Dataset

        params:
          - csv_file: Path to CSV
          - file_col: Column corresponding to file name
          - lbl_col: Column corresponding to label column
          - text2lbl: Numerical representation of labels
          - transforms: Transforms to apply to imgs
        """

        self.csv = pd.read_csv(csv_file)
        self.file_col = file_col
        self.lbl_col = lbl_col
        self.labels = text2lbl
        self.transforms = transforms

    def __len__(self):
        """
        Returns the length of dataset
        """

        return len(self.csv)

    def __getitem__(self, idx):
        """
        Get item from dataset

        params:
          - idx: Index of data record to get.
        """

        img = cv2.imread(self.csv.loc[idx][self.file_col])

        img = img.astype(np.float).transpose(2, 1, 0)
        label = self.csv.loc[idx][self.lbl_col]

        # Convert to tensor and apply transforms
        img = torch.from_numpy(img)
        if self.transforms:
            img = self.transforms(img)

        sample = {"img": img, "label": label}
        return img, label

In [4]:
data = pd.read_csv("/home/nickolas.littlefield/netstore1/unifesp-x-ray-body-part-classifier/train_merged.csv")
# valid =  np.random.choice(class_labels, size=6, replace=False)
# test = np.random.choice(list(set(class_labels).difference(valid)), size=6, replace=False)

In [5]:
test = np.array([16, 10,  2,  0, 13])

In [6]:
import learn2learn as l2l
from learn2learn.data import MetaDataset, FilteredMetaDataset, TaskDataset

img_size = 224
transforms_list = transforms.Compose([transforms.Resize((img_size, img_size))])


test_meta = FilteredMetaDataset(
    XRayDataset("/home/nickolas.littlefield/netstore1/unifesp-x-ray-body-part-classifier/train_merged.csv", 
                "fname", "Target", labels, transforms=transforms_list), test
)

/tmp/ipykernel_1310192/1417516339.py:64: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  img = img.astype(np.float).transpose(2, 1, 0)


In [7]:
nway=5
shot=1

test_query = shot
test_transforms = [
    NWays(test_meta, nway),
    KShots(test_meta, test_query + shot),
    LoadData(test_meta),
    RemapLabels(test_meta),
]
test_tasks = l2l.data.TaskDataset(test_meta, task_transforms=test_transforms, num_tasks=1000)

In [8]:
class PrototypicalNetwork(nn.Module):

    def __init__(self, x_dim, hid_dim, z_dim):
        super().__init__()
        self.encoder = nn.Sequential(
            self.__conv_block(x_dim, hid_dim),
            self.__conv_block(hid_dim, hid_dim),
            self.__conv_block(hid_dim, hid_dim),
            self.__conv_block(hid_dim, z_dim)
        )

    def __conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
    
    def forward(self, x):
        x = self.encoder(x)
        return x.view(x.size(0), -1)


In [9]:
def pairwise_distances_logits(a, b):
    n = a.shape[0]
    m = b.shape[0]
    logits = -((a.unsqueeze(1).expand(n, m, -1) -
                b.unsqueeze(0).expand(n, m, -1))**2).sum(dim=2)
    return logits


def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)

In [10]:
import torch.functional as F
def fast_adapt(model, batch, ways, shot, query_num, metric=None, device=None):
    if metric is None:
        metric = pairwise_distances_logits
    if device is None:
        device = model.device()
    data, labels = batch
    data = data.to(device)
    labels = labels.to(device)
    n_items = shot * ways

    # Sort data samples by labels
    # TODO: Can this be replaced by ConsecutiveLabels ?
    sort = torch.sort(labels)
    data = data.squeeze(0)[sort.indices].squeeze(0)
    labels = labels.squeeze(0)[sort.indices].squeeze(0)

    # Compute support and query embeddings
    embeddings = model(data.float())
    support_indices = np.zeros(data.size(0), dtype=bool)
    selection = np.arange(ways) * (shot + query_num)
    for offset in range(shot):
        support_indices[selection + offset] = True
    query_indices = torch.from_numpy(~support_indices)
    support_indices = torch.from_numpy(support_indices)
    support = embeddings[support_indices].float()
    support = support.reshape(ways, shot, -1).mean(dim=1)
    query = embeddings[query_indices].float()
    labels = labels[query_indices].long()

    logits = pairwise_distances_logits(query, support)
    loss = nn.CrossEntropyLoss()(logits, labels)
    acc = accuracy(logits, labels)
    return loss, acc


In [11]:
test_loader = DataLoader(test_tasks, shuffle=True)
model = torch.load("xray_proto_1shot_5-4-2023.pkl")

In [12]:
accuracies = []

for batch in tqdm(test_loader):
    loss, acc = fast_adapt(model,
                           batch,
                           nway,
                           shot,
                           test_query,
                           metric=pairwise_distances_logits,
                           device=device)
   
    accuracies.append(acc)

  0%|          | 0/1000 [00:00<?, ?it/s]/tmp/ipykernel_1310192/1417516339.py:64: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  img = img.astype(np.float).transpose(2, 1, 0)
100%|██████████| 1000/1000 [19:28<00:00,  1.17s/it]


In [13]:
torch.tensor(accuracies).mean()

tensor(0.3590)

In [14]:
torch.tensor(accuracies).std()

tensor(0.2057)

In [15]:
X, y = test_tasks.sample()

/tmp/ipykernel_1310192/1417516339.py:64: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  img = img.astype(np.float).transpose(2, 1, 0)


In [16]:
y

tensor([4, 4, 0, 0, 2, 2, 1, 1, 3, 3])

In [17]:
nway=5
shot=3

test_query = shot
test_transforms = [
    NWays(test_meta, nway),
    KShots(test_meta, test_query + shot),
    LoadData(test_meta),
    RemapLabels(test_meta),
]
test_tasks = l2l.data.TaskDataset(test_meta, task_transforms=test_transforms, num_tasks=1000)

In [18]:
test_loader = DataLoader(test_tasks, shuffle=True)
model = torch.load("xray_proto_3shot_5-4-2023.pkl")

In [19]:
accuracies = []

for batch in tqdm(test_loader):
    loss, acc = fast_adapt(model,
                           batch,
                           nway,
                           shot,
                           test_query,
                           metric=pairwise_distances_logits,
                           device=device)
   
    accuracies.append(acc)

  0%|          | 0/1000 [00:00<?, ?it/s]/tmp/ipykernel_1310192/1417516339.py:64: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  img = img.astype(np.float).transpose(2, 1, 0)
100%|██████████| 1000/1000 [57:55<00:00,  3.48s/it]


In [20]:
torch.tensor(accuracies).mean()

tensor(0.4835)

In [21]:
torch.tensor(accuracies).std()

tensor(0.1322)

In [22]:
nway=5
shot=5

test_query = shot
test_transforms = [
    NWays(test_meta, nway),
    KShots(test_meta, test_query + shot),
    LoadData(test_meta),
    RemapLabels(test_meta),
]
test_tasks = l2l.data.TaskDataset(test_meta, task_transforms=test_transforms, num_tasks=1000)

In [23]:
test_loader = DataLoader(test_tasks, shuffle=True)
model = torch.load("xray_proto_5shot_5-4-2023.pkl")

In [24]:
accuracies = []

for batch in tqdm(test_loader):
    loss, acc = fast_adapt(model,
                           batch,
                           nway,
                           shot,
                           test_query,
                           metric=pairwise_distances_logits,
                           device=device)
   
    accuracies.append(acc)

  0%|          | 0/1000 [00:00<?, ?it/s]/tmp/ipykernel_1310192/1417516339.py:64: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  img = img.astype(np.float).transpose(2, 1, 0)
100%|██████████| 1000/1000 [1:37:28<00:00,  5.85s/it]


In [25]:
np.unique(data.Target, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21]),
 array([ 92,  54,  81, 727,   9,  25,  70,  16,  13,  73,  25, 103,  19,
         84, 120,  69,  40,  23,  10,   7,  15,  63]))

In [26]:
torch.tensor(accuracies).mean()

tensor(0.6152)

In [27]:
torch.tensor(accuracies).std()

tensor(0.0994)